In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from multiprocessing import Pool, cpu_count

%matplotlib inline

# Import Data From HDF5 File

In [2]:
store = pd.HDFStore('/home/kelleyld/OTB.h5')

In [3]:
games = store.get('games')

In [4]:
games.head()

,GameID,Outcome,WhiteName,BlackName,WhiteElo,BlackElo,Date
0,1,w,"îwicarz, Maciej","Grossmann, Ryszard",NaN,2205.0,2005-07-01
1,2,b,"îwicarz, Maciej","Starostits, Ilmars",NaN,2450.0,2005-06-29
2,3,b,"OEwicarz, Maciej","Zawadzki, Stanis=/+aw",NaN,NaN,2005-07-02
3,4,w,"Åkesson, Joel","Berg, Emanuel",2333.0,2495.0,2004-01-12
4,5,s,"Åkesson, Joel","Cioara, Andrei-Nestor",2360.0,2383.0,2004-06-20


In [5]:
moves = store.get('moves')

In [6]:
moves.head()

,GameID,Turn,Ply,MovedPiece,SourceCol,SourceRow,DestCol,DestRow,CapturedPiece,PromotionPiece,CheckState,CastleSide
0,1,w,1,P,4,1,4,3,NaN,NaN,NaN,NaN
1,1,b,2,p,3,6,3,4,NaN,NaN,NaN,NaN
2,1,w,3,P,4,3,3,4,p,NaN,NaN,NaN
3,1,b,4,q,3,7,3,4,P,NaN,NaN,NaN
4,1,w,5,N,1,0,2,2,NaN,NaN,NaN,NaN


# Create Opening Moves Table

In [7]:
basic_moves = moves.loc[:, ['GameID', 
                            'Ply', 
                            'MovedPiece',
                            'SourceCol', 
                            'SourceRow', 
                            'DestRow', 
                            'DestCol']]

In [8]:
basic_moves.head()

,GameID,Ply,MovedPiece,SourceCol,SourceRow,DestRow,DestCol
0,1,1,P,4,1,3,4
1,1,2,p,3,6,4,3
2,1,3,P,4,3,4,3
3,1,4,q,3,7,4,3
4,1,5,N,1,0,2,2


Create list of dataframes for each of the first moves:

In [9]:
opening_moves = [basic_moves[basic_moves['Ply'] == ply] for ply in range(1, 11)]

Drop the redundant `Ply` column:

In [10]:
for opening_move in opening_moves: 
    opening_move.drop('Ply', axis=1, inplace=True)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Rename the columns to avoid name collisions:

In [11]:
for index, opening_move in enumerate(opening_moves):
    suffix = str(index + 1)
    opening_move.columns = ['GameID', 
                            'MovedPiece' + suffix, 
                            'SourceCol' + suffix, 
                            'SourceRow' + suffix, 
                            'DestRow' + suffix, 
                            'DestCol' + suffix]

Merge all of these dataframes by `GameID`:

In [12]:
partial_book = opening_moves[0]
for index, opening_move in enumerate(opening_moves[1:]):
    partial_book = partial_book.merge(opening_move, how='outer', on='GameID')
opening_book = partial_book.dropna()

In [13]:
opening_book.head()

,GameID,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,...,MovedPiece9,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10
0,1,P,4,1,3,4,p,3,6,4,...,B,5,0,3,2,n,6,7,5,5
1,2,P,4,1,3,4,p,2,6,4,...,B,5,0,4,1,b,2,7,6,3
2,3,N,6,0,2,5,p,3,6,4,...,N,1,0,1,3,n,6,7,5,5
3,4,P,3,1,3,3,n,6,7,5,...,N,1,0,2,2,p,3,6,4,3
4,5,P,3,1,3,3,p,4,6,5,...,P,0,1,2,0,b,1,3,1,3


# Group With Result Data

In [14]:
game_results = games.loc[:, ['GameID', 'Outcome']]
game_results.index = game_results['GameID']
game_results = game_results.drop('GameID', axis=1)
game_results.head()

,Outcome
GameID,
1,w
2,b
3,b
4,w
5,s


In [15]:
results_book = opening_book.merge(game_results, how='outer', left_index=True, right_index=True)

In [16]:
results_book = results_book.drop('GameID', axis=1)

In [17]:
results_book.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,SourceCol9,SourceRow9,DestRow9,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Outcome
0,P,4,1,3,4,p,3,6,4,3,...,5,0,3,2,n,6,7,5,5,NaN
1,P,4,1,3,4,p,2,6,4,2,...,5,0,4,1,b,2,7,6,3,w
2,N,6,0,2,5,p,3,6,4,3,...,1,0,1,3,n,6,7,5,5,b
3,P,3,1,3,3,n,6,7,5,5,...,1,0,2,2,p,3,6,4,3,b
4,P,3,1,3,3,p,4,6,5,4,...,0,1,2,0,b,1,3,1,3,w


# Grouping Table

In [19]:
def agg_results(results):
    stalemates = results[results['Outcome'] == 's'].shape[0]
    white_wins = results[results['Outcome'] == 'w'].shape[0]
    black_wins = results[results['Outcome'] == 'b'].shape[0]
    total = stalemates + white_wins + black_wins
    
    if total == 0:
        return pd.DataFrame()
    else: 
        ret_df = pd.DataFrame(results.iloc[0, :]).T
        ret_df['Stalemates'] = stalemates / total
        ret_df['White Wins'] = white_wins / total
        ret_df['Black Wins'] = black_wins / total
        ret_df['Frequency'] =  total
        return ret_df

In [20]:
grouping_columns = list(results_book.columns.drop('Outcome').values)
grouped = results_book.groupby(grouping_columns)

In [21]:
with Pool(cpu_count() - 8) as pool:
    mapped = pool.map(agg_results, [group for _, group in grouped])
dirty_outcomes = pd.concat(mapped)
dirty_outcomes.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Outcome,Stalemates,White Wins,Black Wins,Frequency
1493003,N,1,0,2,0,n,1,7,5,0,...,n,1,3,5,2,w,0.0,1.0,0.0,1
1751358,N,1,0,2,0,n,1,7,5,0,...,n,5,3,2,7,b,0.0,0.0,1.0,1
1895044,N,1,0,2,0,n,1,7,5,0,...,p,3,6,4,3,b,0.0,0.0,1.0,1
4734031,N,1,0,2,0,n,1,7,5,0,...,b,5,7,2,0,s,1.0,0.0,0.0,1
5552181,N,1,0,2,0,n,6,7,5,5,...,p,1,6,5,1,w,0.0,1.0,0.0,1


# Clean and Sort Results

In [22]:
goal_columns = grouping_columns + ['Black Wins', 'White Wins', 'Stalemates', 'Frequency']
outcomes = dirty_outcomes.reset_index().loc[:, goal_columns]
outcomes['Frequency'] = outcomes['Frequency'].astype('int')
outcomes.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency
0,N,1,0,2,0,n,1,7,5,0,...,2,n,1,3,5,2,0.0,1.0,0.0,1
1,N,1,0,2,0,n,1,7,5,0,...,5,n,5,3,2,7,1.0,0.0,0.0,1
2,N,1,0,2,0,n,1,7,5,0,...,3,p,3,6,4,3,1.0,0.0,0.0,1
3,N,1,0,2,0,n,1,7,5,0,...,2,b,5,7,2,0,0.0,0.0,1.0,1
4,N,1,0,2,0,n,6,7,5,5,...,4,p,1,6,5,1,0.0,1.0,0.0,1


In [23]:
sorted_outcomes = outcomes.sort_values('Frequency', ascending=False)

In [24]:
sorted_outcomes.head()

,MovedPiece1,SourceCol1,SourceRow1,DestRow1,DestCol1,MovedPiece2,SourceCol2,SourceRow2,DestRow2,DestCol2,...,DestCol9,MovedPiece10,SourceCol10,SourceRow10,DestRow10,DestCol10,Black Wins,White Wins,Stalemates,Frequency
491416,P,4,1,3,4,p,2,6,4,2,...,2,p,0,6,5,0,0.306617,0.390478,0.302905,146290
594900,P,4,1,3,4,p,4,6,4,4,...,6,b,5,7,6,4,0.305951,0.392637,0.301412,80405
491421,P,4,1,3,4,p,2,6,4,2,...,2,p,6,6,5,6,0.307274,0.390700,0.302026,57164
480899,P,4,1,3,4,p,2,6,4,2,...,2,p,4,6,4,4,0.305496,0.388587,0.305917,45143
274089,P,3,1,3,3,n,6,7,5,5,...,5,k,4,0,0,6,0.308641,0.390539,0.300820,41942


In [25]:
for column in grouping_columns:
    sorted_outcomes[column] = sorted_outcomes[column].astype('category')

# Write to HDF5

In [26]:
store.put('openings', results_book, format='table')

In [27]:
store.put('opening_results', sorted_outcomes, format='table')